# Implementation of the Eigen value based Loss Function
This notebook contains the implementation of eigen value based loss function with examples.

## Imports
We first import all necessary packages and modules

In [1]:
import numpy as np
import networkx as nx
import torch
import  random
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Construction of Graphs from one hot vectors
The outputs from CNN are formatted as one-hot encoded vectors of dimension 5 (since we are dealing with prediction of 5 mutations). Let us call this predicted one hot vector as $y_{pred}$ and its corresponding ground truth as $y_{true}$. We can create graphs from $y_{pred}$ and $y_{true}$. If we consider each node as a mutation, there will be 5 nodes. All mutations with same value will be connected with each other. For eg. if $y_{pred} = [1,0,1,1,0]$ where $y_{pred}[0]$ indicates absence(0)/presence(1) of mutation $0$, then node indicating 'mutation_0' will be connected to nodes indicating 'mutation_2' and 'mutation_3', while 'mutation_1' and 'mutation_4' will be connected. We will connect similarly for all other mutations.
Let us consider the graphs constructed from $y_{pred}$ and $y_{true}$ as $G_{pred}$ and $G_{true}$.

### Sample vector generation
Let us create some random one vectors indicating the presence/absence of the mutations

In [2]:
num_samples = 10 # Number of samples
y_pred = torch.randint(0,2,(num_samples,5), dtype=float, requires_grad=True, device=DEVICE)
y_true = torch.randint(0,2,(num_samples,5), dtype=float, requires_grad=True, device=DEVICE)
print("Simulated Predicted vectors: ", y_pred)
print("Simulated Ground Truth vectors: ", y_true)
print("Using device: ", y_pred.get_device())

Simulated Predicted vectors:  tensor([[0., 0., 1., 1., 1.],
        [1., 1., 1., 1., 0.],
        [1., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0.],
        [1., 1., 0., 1., 0.],
        [1., 0., 1., 1., 0.],
        [0., 1., 1., 0., 0.],
        [0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)
Simulated Ground Truth vectors:  tensor([[1., 1., 1., 0., 0.],
        [1., 0., 0., 0., 1.],
        [1., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [1., 1., 0., 1., 0.],
        [0., 1., 1., 0., 1.],
        [0., 0., 0., 1., 1.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)
Using device:  0


### Graph building
Now let us create graphs from `y_true` and `y_pred`

In [37]:
# x = torch.tensor([1,0,1,1,0], dtype=float, requires_grad=True)
# y = 1 - x
# z = torch.matmul(torch.reshape(y, (-1, 1)), torch.reshape(x, (1, -1)))  # y_trans (5, 1) * x (1, 5) = z (5, 5)
# print(z)
# p = torch.flip(z, (0, ))
# p

tensor([[0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0.]], dtype=torch.float64, grad_fn=<MmBackward0>)


tensor([[0., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0.]], dtype=torch.float64, grad_fn=<FlipBackward0>)

In [39]:
# x = torch.tensor([1,0,1,1,0], dtype=float, requires_grad=True)
# y = 1 - x
# z = torch.matmul(torch.reshape(x, (-1, 1)), torch.reshape(y, (1, -1)))  # x (5, 1) * y_trans (1, 5) = z (5, 5)
# print(z)
# p = torch.flip(z, (0, 1))
# p

tensor([[0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0.]], dtype=torch.float64, grad_fn=<MmBackward0>)


tensor([[0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0.]], dtype=torch.float64, grad_fn=<FlipBackward0>)

In [4]:
def build_graph(x: torch.Tensor) -> torch.Tensor:
    y = 1 - x
    x_list = list(x.detach().cpu().numpy())
    for idx, mutation_vector in enumerate(x_list):
        if mutation_vector[0] == 0:
            mat = torch.select(y, 0, idx)
        else:
            mat = torch.select(x, 0, idx)
        for mutation_value in mutation_vector[1:]:
            if mutation_value == 0:
                mat = torch.row_stack((mat, torch.select(y, 0, idx)))
            else:
                mat = torch.row_stack((mat, torch.select(x, 0, idx)))
        if idx == 0:
            super_mat = mat
        else:
            super_mat = torch.row_stack((super_mat, mat))
    return torch.reshape(super_mat, (10,5,5,))

graph_pred = build_graph(y_pred)
graph_true = build_graph(y_true)

## Graph Similarity calculation based on Eigen values
### Calculate the Laplacians of $G_{pred}$ and $G_{true}$

In [5]:
def calculate_laplacian(adjacency_matrix: torch.Tensor) -> torch.Tensor:
    # Calculate the degree matrix
    degree_matrix = torch.diag(torch.sum(adjacency_matrix, dim=1)-1)

    # Calculate the Laplacian matrix
    laplacian_matrix = degree_matrix - adjacency_matrix

    return laplacian_matrix

lap_pred = calculate_laplacian(graph_pred)
lap_true = calculate_laplacian(graph_true)

### Calculate the eigen values using `torch.linalg.eigvals()` for each laplacian

In [6]:
def select_best_eigen_values(eigen_values: torch.Tensor, threshold: float) -> int:
    total_eig_sum = torch.sum(eigen_values, dim=1)    # Sum the eigen values of each data
    num_values = eigen_values.shape[1]
    for k in range(1, num_values+1):  # looping in range 1 to number of eigen values for each data
        column_eig_vals = eigen_values[:, 0:k]    # selecting the first k values from each eigen value
        temp_eig_sum = torch.sum(column_eig_vals, dim=1)
        energy = torch.abs(torch.mean(temp_eig_sum/total_eig_sum)).item()
        if energy > threshold:
            return k
    return k

def eigen_value_similarity(l_pred: torch.Tensor, l_true:torch.Tensor) -> torch.Tensor:
    pred_eigvals = torch.linalg.eigvals(l_pred)
    true_eigvals = torch.linalg.eigvals(l_true)
    # print("Predicted Eigen values:", pred_eigvals)
    # print("Actual Eigen values:", true_eigvals)
    k_pred = select_best_eigen_values(pred_eigvals, 0.85)
    k_true = select_best_eigen_values(true_eigvals, 0.85)
    min_k = min(k_true, k_pred)
    distance = torch.norm(pred_eigvals[:, 0:min_k] - true_eigvals[0:min_k])
    return distance
eigen_loss = eigen_value_similarity(lap_pred, lap_true)
print("Eigen Loss:", eigen_loss)

Eigen Loss: tensor(37.0705, device='cuda:0', dtype=torch.float64,
       grad_fn=<LinalgVectorNormBackward0>)


In [7]:
print(y_pred.grad)
eigen_loss.backward()
print(y_pred.grad)

None
tensor([[ 6.3831,  0.0474,  0.0418,  0.0418,  0.0418],
        [ 0.0981, 19.5768,  0.0981,  0.0981, -0.4456],
        [-0.0474, -0.0474, -5.9982, -0.0418, -0.0418],
        [ 0.3377,  0.2969,  0.2969, -5.5309,  0.2969],
        [-0.1263, -0.1263, -0.1434, -0.1263,  6.0187],
        [ 0.0418,  0.0474,  0.0418,  0.0418,  0.0474],
        [-0.2078, -0.2349, -0.2349, -0.2078, -0.2078],
        [-0.0418, -0.0474, -0.0418, -0.0418, -0.0474],
        [ 0.0364,  0.0535,  0.0364,  0.0364,  0.0364],
        [ 0.5255,  0.3409,  0.3409,  0.3409,  0.3409]], device='cuda:0',
       dtype=torch.float64)
